<a href="https://colab.research.google.com/github/tejasspawar/MR-AL-I/blob/main/ME781_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F
import torch.utils.data as data
import random
import math
from torch.nn.init import kaiming_uniform_, kaiming_normal_

# import sys
# sys.path.append('../input/iterative-stratification/iterative-stratification-master')
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import numpy as np

# Data Loader

In [ ]:
class MRDataset(data.Dataset):
    def __init__(self, root_dir, task, plane, train=True, transform=None):
        super().__init__()
        self.task = task
        self.plane = plane
        self.root_dir = root_dir
        self.train = train
        if self.train:
            self.folder_path = self.root_dir + 'train/{0}/'.format(plane)
            self.records = pd.read_csv(
                self.root_dir + 'train-{0}.csv'.format(task), header=None, names=['id', 'label'])
        else:
            transform = None
            self.folder_path = self.root_dir + 'valid/{0}/'.format(plane)
            self.records = pd.read_csv(
                self.root_dir + 'valid-{0}.csv'.format(task), header=None, names=['id', 'label'])

        self.records['id'] = self.records['id'].map(
            lambda i: '0' * (4 - len(str(i))) + str(i))
        self.paths = [self.folder_path + filename +
                      '.npy' for filename in self.records['id'].tolist()]
        self.labels = self.records['label'].tolist()

        self.transform = transform
    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        array = np.load(self.paths[index])
        label = self.labels[index]
        if label == 1:
            label = torch.FloatTensor([[0, 1]])
        elif label == 0:
            label = torch.FloatTensor([[1, 0]])

        if self.transform:
            array = self.transform(array)

        return array, label

# CNN Architechture

Reference: https://arxiv.org/abs/2005.02706

In [ ]:
%%file elnet.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import math
import os
from torch.nn.init import kaiming_uniform_, kaiming_normal_

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def make_deterministic(seed):

    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def weight_init(m, seed=2, init_type='uniform'):

    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):

        if init_type == 'normal':
            kaiming_normal_(m.weight)
        else:
            kaiming_uniform_(m.weight, a=math.sqrt(5))
    else:
        raise TypeError("cannnot initialize such weights")


def get_norm_layer(channels, norm_type='layer'):
    if norm_type == 'instance':
        layer = nn.GroupNorm(channels, channels)
    elif norm_type == 'batch':
        layer = nn.BatchNorm2d(channels)
    else:
        layer = nn.GroupNorm(1, channels)  # layer norm by default

    return layer


def conv_block(channels, kernel_size, dilation=1, repeats=2, normalization='layer', seed=2, init_type='uniform'):
    """
    :param channels: the input channel amount (same for output)
    :param kernel_size: 2D convolution kernel
    :param dilation: the dialation for the kernels of a conv block
    :param padding: amount of padding
    :param repeats: amount of repeats before added with identity
    :param normalization: the type of multi-slice normalization used
    :param seed: which seed of initial weights to use
    :param init_type: which type of Kaiming Init to use
    :return: nn.Sequential(for the given block)
    """

    conv_list = nn.ModuleList([])

    for i in range(repeats):
        conv2d = nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=kernel_size,
                           dilation=dilation, stride=1, bias=False,
                           padding=(kernel_size + ((dilation - 1) * (kernel_size - 1))) // 2)
        weight_init(conv2d, seed=seed, init_type=init_type)
        conv_list.append(conv2d)

        #   Instance Normalization and Layer Normalization are just variations of Group Normalization
        #   https://pytorch.org/docs/stable/nn.html#groupnorm

        conv_list.append(get_norm_layer(channels, normalization))
        conv_list.append(nn.ReLU())

    return nn.Sequential(*conv_list)


def get_antialiasing_filter(kernel_size):
    """Get an integer specifying the 2D kernel size >>> returns a (1 x 1 x kernel_size x kernel_size)"""

    kernel_dict = {
        1: [[[[1.]]]],

        2: [[[[0.2500, 0.2500],
              [0.2500, 0.2500]]]],

        3: [[[[0.0625, 0.1250, 0.0625],
              [0.1250, 0.2500, 0.1250],
              [0.0625, 0.1250, 0.0625]]]],

        4: [[[[0.0156, 0.0469, 0.0469, 0.0156],
              [0.0469, 0.1406, 0.1406, 0.0469],
              [0.0469, 0.1406, 0.1406, 0.0469],
              [0.0156, 0.0469, 0.0469, 0.0156]]]],

        5: [[[[0.0039, 0.0156, 0.0234, 0.0156, 0.0039],
              [0.0156, 0.0625, 0.0938, 0.0625, 0.0156],
              [0.0234, 0.0938, 0.1406, 0.0938, 0.0234],
              [0.0156, 0.0625, 0.0938, 0.0625, 0.0156],
              [0.0039, 0.0156, 0.0234, 0.0156, 0.0039]]]],

        6: [[[[0.0010, 0.0049, 0.0098, 0.0098, 0.0049, 0.0010],
              [0.0049, 0.0244, 0.0488, 0.0488, 0.0244, 0.0049],
              [0.0098, 0.0488, 0.0977, 0.0977, 0.0488, 0.0098],
              [0.0098, 0.0488, 0.0977, 0.0977, 0.0488, 0.0098],
              [0.0049, 0.0244, 0.0488, 0.0488, 0.0244, 0.0049],
              [0.0010, 0.0049, 0.0098, 0.0098, 0.0049, 0.0010]]]],

        7: [[[[0.0002, 0.0015, 0.0037, 0.0049, 0.0037, 0.0015, 0.0002],
              [0.0015, 0.0088, 0.0220, 0.0293, 0.0220, 0.0088, 0.0015],
              [0.0037, 0.0220, 0.0549, 0.0732, 0.0549, 0.0220, 0.0037],
              [0.0049, 0.0293, 0.0732, 0.0977, 0.0732, 0.0293, 0.0049],
              [0.0037, 0.0220, 0.0549, 0.0732, 0.0549, 0.0220, 0.0037],
              [0.0015, 0.0088, 0.0220, 0.0293, 0.0220, 0.0088, 0.0015],
              [0.0002, 0.0015, 0.0037, 0.0049, 0.0037, 0.0015, 0.0002]]]]

    }

    if kernel_size in kernel_dict:
        return torch.Tensor(kernel_dict[kernel_size])
    else:
        raise ValueError('Unrecognized kernel size')


class BlurPool(nn.Module):

    def __init__(self, channels, stride, filter_size=3):
        super(BlurPool, self).__init__()
        self.channels = channels  # same input and output channels
        self.filter_size = filter_size
        self.stride = stride
        '''Kernel is a 1x5x5 kernel'''

        # repeat tensor from (1 x 1 x fs x fs) >>> (channels x 1 x fs x fs)
        self.kernel = nn.Parameter(get_antialiasing_filter(filter_size).repeat(self.channels, 1, 1, 1),
                                   requires_grad=False)

    def forward(self, x):
        """
        x is a tensor of dimension (batch, in_channels, height, width)
        - assume same input and output channels, and groups = 1
        - CURRENTLY DON'T SUPPORT PADDING
        """

        y = F.conv2d(input=x, weight=self.kernel, stride=self.stride, groups=self.channels)
        return y

    def to(self, dtype):
        self.kernel = self.kernel.to(dtype)
        return self



class ELNet(nn.Module):

    def __init__(self, **kwargs):

        super().__init__()

        self.K = kwargs.get('K', 4)  # default K for ELNet
        self.norm_type = kwargs.get('norm_type', 'layer')  # default multi-slice normalization
        self.aa_filter = kwargs.get('aa_filter_size', 5)  # default aa-filter configuration
        self.weight_init_type = kwargs.get('weight_init_type', 'normal')  # type of weight initialization
        self.seed = kwargs.get('seed', 2)  # default seed for initialization
        self.num_classes = kwargs.get('num_classes', 2)  # number of classes for ELNet

        make_deterministic(self.seed)

        if isinstance(self.aa_filter, int):
            aa_filter_size = [self.aa_filter] * 5
            print(aa_filter_size)

        self.channel_config = [4 * self.K, 8 * self.K, 16 * self.K, 16 * self.K, 16 * self.K]

        self.conv_1 = nn.Conv2d(1, self.channel_config[0], kernel_size=7, stride=2, padding=3, bias=False)
        self.downsample_1 = BlurPool(channels=self.channel_config[0], stride=2, filter_size=aa_filter_size[0])
        self.norm_1 = get_norm_layer(self.channel_config[0], norm_type=self.norm_type)

        self.conv_2 = conv_block(self.channel_config[0], kernel_size=5, repeats=2, normalization=self.norm_type)
        self.conv_2_to_3 = nn.Conv2d(self.channel_config[0], self.channel_config[1], kernel_size=5, stride=1, padding=2,
                                     bias=False)
        self.downsample_2 = BlurPool(channels=self.channel_config[1], stride=2, filter_size=aa_filter_size[1])

        self.conv_3 = conv_block(self.channel_config[1], kernel_size=3, repeats=2, normalization=self.norm_type)
        self.conv_3_to_4 = nn.Conv2d(self.channel_config[1], self.channel_config[2], kernel_size=3, stride=1, padding=1,
                                     bias=False)
        self.downsample_3 = BlurPool(channels=self.channel_config[2], stride=2, filter_size=aa_filter_size[2])

        self.conv_4 = conv_block(self.channel_config[2], kernel_size=3, repeats=1, normalization=self.norm_type)
        self.conv_4_to_5 = nn.Conv2d(self.channel_config[2], self.channel_config[3], kernel_size=3, stride=1, padding=1,
                                     bias=False)
        self.downsample_4 = BlurPool(channels=self.channel_config[3], stride=2, filter_size=aa_filter_size[3])

        self.conv_5 = conv_block(self.channel_config[3], kernel_size=3, repeats=1, normalization=self.norm_type)
        self.conv_5_to_6 = nn.Conv2d(self.channel_config[3], self.channel_config[4], kernel_size=3, stride=1, padding=1,
                                     bias=False)
        self.downsample_5 = BlurPool(channels=self.channel_config[4], stride=2, filter_size=aa_filter_size[4])

        self.max_pool = nn.AdaptiveMaxPool1d(1)

        self.feature_dp = nn.Dropout(0.7)

        self.fc = nn.Linear(self.channel_config[4], self.num_classes)

        weight_init(self.conv_1, self.seed, self.weight_init_type)
        weight_init(self.conv_2_to_3, self.seed, self.weight_init_type)
        weight_init(self.conv_3_to_4, self.seed, self.weight_init_type)
        weight_init(self.conv_4_to_5, self.seed, self.weight_init_type)
        weight_init(self.conv_5_to_6, self.seed, self.weight_init_type)
        weight_init(self.fc, self.seed, self.weight_init_type)

    def feature_extraction(self, x):
        x = x.permute(1,0, 2, 3)
        #print(x.size())

        if self.training:

            x = self.downsample_1(F.relu(self.norm_1(self.conv_1(x))))

            x = x + self.conv_2(x)  # skip connection (survival rate 1 for first skip)
            x = self.downsample_2(F.relu(self.conv_2_to_3(x)))

            x = x + self.conv_3(x)
            x = self.downsample_3(F.relu(self.conv_3_to_4(x)))

            x = x + self.conv_4(x)
            x = self.downsample_4(F.relu(self.conv_4_to_5(x)))

            x = x + self.conv_5(x)
            x = self.downsample_5(F.relu(self.conv_5_to_6(x)))

        else:  # evaluation mode

            x = self.downsample_1(F.relu(self.norm_1(self.conv_1(x)), inplace=True))

            x += self.conv_2(x)  # in-place skip connection (not suitable for training pass)
            x = self.downsample_2(F.relu(self.conv_2_to_3(x), inplace=True))

            x += self.conv_3(x)
            x = self.downsample_3(F.relu(self.conv_3_to_4(x), inplace=True))

            x += self.conv_4(x)  # skip connection
            x = self.downsample_4(F.relu(self.conv_4_to_5(x), inplace=True))

            x += self.conv_5(x)  # skip connection
            x = self.downsample_5(F.relu(self.conv_5_to_6(x), inplace=True))

        feats = nn.AdaptiveMaxPool2d(1)(x)  # get [sx16Kx1x1]
        feats = self.feature_dp(feats)  # performs feature-wise dropout

        return feats

    def forward(self, x):
        feats = self.feature_extraction(x)  # get [sx16Kx1x1]
        feats = feats.squeeze(3)
        feats = feats.permute(2, 1, 0)  # [1x16Kxs]

        # classifier
        feats = self.max_pool(feats).squeeze(2)  # [1x16K]
        scores = self.fc(feats)
        return scores

Overwriting elnet.py


#Predictor

In [ ]:
!pwd

/content


In [ ]:
!ls

elnet.py  sample_data


In [ ]:
import os
import torch

from elnet import *
import numpy as np

In [ ]:
def check(input_img):
    
    print(" your image is : " + input_img)
   
    input_img1=f'../app/images/{input_img}'
    img = np.load(input_img1)
    img=img/255
    img=torch.Tensor(img)
    img=img.repeat(1,1,1,1)
    print(img.size())
    l=['abnormal','meniscus']
    model_path=[]
    elNet=[]
    output=[]
    probas=[]
    label={}
    model_name = ['model_patience_5 _gamma_2n_abnormal_axial_val_auc_0.8636_train_auc_0.9016_epoch_39.pth','model_patience_5 _gamma_2n_meniscus_coronal_val_auc_0.7868_train_auc_0.7720_epoch_32.pth']     
    for i in range(0,2):
        model_name1=model_name[i]
        m = f'../app/models/{model_name1}'
        model_path.append(m)
        elNet.append(torch.load(model_path[i],map_location='cpu'))
        _ = elNet[i].eval()
        x=elNet[i].forward(img.float())
        output.append(x)
        y=torch.sigmoid(x)
        probas.append(y)
        label[l[i]]=(probas[i][0][1].item())
    print(label)        
    return label

# Training Pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls

drive  elnet,py  elnet.py  __pycache__	sample_data


In [ ]:
cd drive/MyDrive/Knee_MRI

/content/drive/MyDrive/Knee_MRI


In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 124 kB 4.3 MB/s 


In [ ]:
import shutil
import os
import time
from datetime import datetime
import argparse
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
# from torchsample.transforms import RandomRotate, RandomTranslate, RandomFlip, ToTensor, Compose, RandomAffine
from torchvision import transforms
import torch.nn.functional as F
from tensorboardX import SummaryWriter

from eldata import MRDataset
import elnet

from sklearn import metrics

In [ ]:
# Insert the directory
import sys
sys.path.insert(0,'/content/drive/MyDrive/Knee_MRI')

# Import your module or file
import elnet

In [ ]:
def train_model(elnet, train_loader, epoch, num_epochs, optimizer, writer, current_lr, log_every=100):
    _ = elnet.train()

    if torch.cuda.is_available():
        elnet.cuda()

    y_preds = []
    y_trues = []
    losses = []

    for i, (image, label) in enumerate(train_loader):
        optimizer.zero_grad()

        if torch.cuda.is_available():
            image = image.cuda()
            label = label.cuda()
            # weight = weight.cuda()

        label = label[0]
        #print(label)
        #weight = weight[0]
        #output=elnet(image)

        prediction = elnet.forward(image.float())

        loss = torch.nn.BCEWithLogitsLoss()(prediction, label)
        loss.backward()
        optimizer.step()

        loss_value = loss.item()
        losses.append(loss_value)

        probas = torch.sigmoid(prediction)

        y_trues.append(int(label[0][1]))
        y_preds.append(probas[0][1].item())

        try:
            auc = metrics.roc_auc_score(y_trues, y_preds)
        except:
            auc = 0.5

        writer.add_scalar('Train/Loss', loss_value,
                          epoch * len(train_loader) + i)
        writer.add_scalar('Train/AUC', auc, epoch * len(train_loader) + i)

        if (i % log_every == 0) & (i > 0):
            print('''[Epoch: {0} / {1} |Single batch number : {2} / {3} ]| avg train loss {4} | train auc : {5} | lr : {6}'''.
                  format(
                      epoch + 1,
                      num_epochs,
                      i,
                      len(train_loader),
                      np.round(np.mean(losses), 4),
                      np.round(auc, 4),
                      current_lr
                  )
                  )

    writer.add_scalar('Train/AUC_epoch', auc, epoch + i)

    train_loss_epoch = np.round(np.mean(losses), 4)
    train_auc_epoch = np.round(auc, 4)
    return train_loss_epoch, train_auc_epoch


def evaluate_model(elnet, val_loader, epoch, num_epochs, writer, current_lr, log_every=20):
    _ = elnet.eval()

    if torch.cuda.is_available():
        elnet.cuda()

    y_trues = []
    y_preds = []
    losses = []

    for i, (image, label) in enumerate(val_loader):

        if torch.cuda.is_available():
            image = image.cuda()
            label = label.cuda()
            #weight = weight.cuda()

        label = label[0]
       # weight = weight[0]

        prediction = elnet.forward(image.float())

        loss = torch.nn.BCEWithLogitsLoss( )(prediction, label)

        loss_value = loss.item()
        losses.append(loss_value)

        probas = torch.sigmoid(prediction)

        y_trues.append(int(label[0][1]))
        y_preds.append(probas[0][1].item())

        try:
            auc = metrics.roc_auc_score(y_trues, y_preds)
        except:
            auc = 0.5

        writer.add_scalar('Val/Loss', loss_value, epoch * len(val_loader) + i)
        writer.add_scalar('Val/AUC', auc, epoch * len(val_loader) + i)

        if (i % log_every == 0) & (i > 0):
            print('''[Epoch: {0} / {1} |Single batch number : {2} / {3} ] | avg val loss {4} | val auc : {5} | lr : {6}'''.
                  format(
                      epoch + 1,
                      num_epochs, 
                      i,
                      len(val_loader),
                      np.round(np.mean(losses), 4),
                      np.round(auc, 4),
                      current_lr
                  )
                  )

    writer.add_scalar('Val/AUC_epoch', auc, epoch + i)

    val_loss_epoch = np.round(np.mean(losses), 4)
    val_auc_epoch = np.round(auc, 4)
    return val_loss_epoch, val_auc_epoch

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [ ]:
def run(task, plane, prefix_name):
  lr= 3e-5
  lr_scheduler='plateau'
  augment=1
  gamma = 4
  epochs= 50
  flush_history=0
  save_model = 1
  patience = 10
  log_every=100
  log_root_folder = "./logs/{0}/{1}/".format(task, plane)
  if flush_history == 1:
          objects = os.listdir(log_root_folder)
          for f in objects:
              if os.path.isdir(log_root_folder + f):
                  shutil.rmtree(log_root_folder + f)

  now = datetime.now()
  logdir = log_root_folder + now.strftime("%Y%m%d-%H%M%S") + "/"
  os.makedirs(logdir)
  writer = SummaryWriter(logdir)
  '''
  def fnc(x):
      return x.repeat(1,1,1,1).permute(0,1,2,3)
  
  def fnc1(x):
      return torch.Tensor(x)
'''
  augmentor = transforms.Compose([
      transforms.Lambda(lambda x: torch.Tensor(x)),
      transforms.RandomRotation(degrees=(-10, 10)),
      #transforms.RandomTranslate([0.11, 0.11]),
      transforms.RandomHorizontalFlip(0.5),
      #transforms.RandomFlip(),
      transforms.RandomAffine(degrees=(10, 20), translate=(0.1, 0.1), scale=(0.8,0.8)),
    
      #transforms.Lambda(lambda x: x.repeat(1, 1, 1, 1).permute(0, 1, 2, 3)),
      ])

  train_dataset = MRDataset('./data/', task,plane, transform=augmentor, train=True)
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=0, drop_last=False)

  validation_dataset = MRDataset( './data/', task, plane, train=False)
  validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=-True, num_workers=0, drop_last=False)

  Elnet = elnet.ELNet()

  if torch.cuda.is_available():
      
      Elnet = Elnet.cuda()

  optimizer = optim.Adam(Elnet.parameters(), lr=lr)

  if lr_scheduler == "plateau":
      scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer, factor=.3, threshold=1e-4, verbose=True)
  elif lr_scheduler == "step":
      scheduler = torch.optim.lr_scheduler.StepLR(
          optimizer, step_size=3, gamma=gamma)

  best_val_loss = float('inf')
  best_val_auc = float(0)

  num_epochs = epochs
  iteration_change_loss = 0
  patience = patience
  log_every = log_every

  t_start_training = time.time()

  for epoch in range(num_epochs):
      current_lr = get_lr(optimizer)

      t_start = time.time()
      
      train_loss, train_auc = train_model(Elnet, train_loader, epoch, num_epochs, optimizer, writer, current_lr, log_every)
      val_loss, val_auc = evaluate_model(Elnet, validation_loader, epoch, num_epochs, writer, current_lr)

      if lr_scheduler == 'plateau':
          scheduler.step(val_loss)
      elif lr_scheduler == 'step':
          scheduler.step()

      t_end = time.time()
      delta = t_end - t_start

      print("train loss : {0} | train auc {1} | val loss {2} | val auc {3} | elapsed time {4} s".format(train_loss, train_auc, val_loss, val_auc, delta))

      iteration_change_loss += 1
      print('-' * 30)

      if val_auc > best_val_auc:
          best_val_auc = val_auc
          if bool(save_model):
              file_name = f'model_{prefix_name}_{task}_{plane}_val_auc_{val_auc:0.4f}_train_auc_{train_auc:0.4f}_epoch_{epoch+1}.pth'
              for f in os.listdir('./data/models/'):
                  if (task in f) and (plane in f) and (prefix_name in f):
                      os.remove(f'./data/models/{f}')
              torch.save(Elnet, f'./data/models/{file_name}')

      if val_loss < best_val_loss:
          best_val_loss = val_loss
          iteration_change_loss = 0

      if iteration_change_loss == patience:
          print('Early stopping after {0} iterations without the decrease of the val loss'.
                format(iteration_change_loss))
          break
  # return best_loss
  t_end_training = time.time()
  print(f'training took {t_end_training - t_start_training} s')

In [ ]:
if __name__ == "__main__":
  run('acl','axial', 'ACL_Axial')

[Epoch: 1 / 50 |Single batch number : 100 / 1130 ]| avg train loss 0.625 | train auc : 0.5667 | lr : 3e-05
[Epoch: 1 / 50 |Single batch number : 200 / 1130 ]| avg train loss 0.5646 | train auc : 0.5547 | lr : 3e-05
[Epoch: 1 / 50 |Single batch number : 300 / 1130 ]| avg train loss 0.5441 | train auc : 0.5515 | lr : 3e-05
[Epoch: 1 / 50 |Single batch number : 400 / 1130 ]| avg train loss 0.5297 | train auc : 0.5671 | lr : 3e-05
[Epoch: 1 / 50 |Single batch number : 500 / 1130 ]| avg train loss 0.5151 | train auc : 0.5678 | lr : 3e-05
[Epoch: 1 / 50 |Single batch number : 600 / 1130 ]| avg train loss 0.4999 | train auc : 0.5882 | lr : 3e-05
[Epoch: 1 / 50 |Single batch number : 700 / 1130 ]| avg train loss 0.5079 | train auc : 0.5967 | lr : 3e-05
[Epoch: 1 / 50 |Single batch number : 800 / 1130 ]| avg train loss 0.5085 | train auc : 0.5981 | lr : 3e-05
[Epoch: 1 / 50 |Single batch number : 900 / 1130 ]| avg train loss 0.5091 | train auc : 0.5985 | lr : 3e-05
[Epoch: 1 / 50 |Single batch 